<a href="https://colab.research.google.com/github/Utsav147/Google-Colab-TF-Practice/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
import keras

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential